# Chapter 12: IP Weighting and Marginal Structural Models

While IP Weighting was initially only introduced as a nonparametric method, under additional assumptions, it will allow us to tackle high-dimensional problems with many covariates and nondichotomous treatments.

## 12.1 The Causal Question

We will use $A$ as a binary flag of smoking cessation and $Y$ as the outcome of weight gain.

Assume we also have 9 covariates $L$ which includes sex, age, race, education, smoking intensity, etc., which we use IP weighting to adjust.

## 12.2 Estimating IP Weights via Modeling

Recall that IP Weighting creates a pseudo-population in which the arrow from the covariates $L$ to the treatment $A$ is removed.

More precisely, the pseudo-population has the following two properties:
1. $A$ and $L$ are statistically independent
2. the mean $E_{ps}[Y|A=a]$ in the pseudo-population equals the standardized mean $\sum_lE[Y|A=a,L=l]Pr[L=l]$ in the actual population.

These properties are true even if conditional exchangeability $Y^a\perp A|L$ does not hold in the actual population.

Now, if conditional exchangeability holds, then
1. the mean of $Y^a$ is the same in both populations
2. unconditional exchangeability holds in the pseudo-population
3. the counter-factual mean $E[Y^a]$ in the actual population is equal to $E_{ps}[Y|A=a]$ in the pseudo-population
4. association is causation in the pseudo-population

Remember that the pseudo-population is created by weighting each individual by the inverse of the propensity score.

The individual-specific IP weights for treatment $A$ are defined as:
$$W^A=\frac{1}{f(A|L)}$$

Previously, we estimated dichotomous $Pr[A=1|L]$ nonparametrically by simply counting how many people were in treated in each stratum of $L$ and then dividing this count by # of individuals in the stratum. This is not realistic for high-dimensional $L$.

To obtain parametric estimates, we fit a logistic regression model. Of-course, there has been a lot of innovation on how to fit these parameter estimates but they are not in the scope of this textbook.

To obtain a 95% CI around the point estimate $\hat\theta_1$, we need a method that takes the IP Weighting into account. The typical go-to method is the robust variance estimator, although they tend to be more conservative.

## 12.3 Stabilized IP Weights

Note that the weights $W^A=1/f(A|L)$ is just a particular example of IP weights with $p=1$, but the numerator can be anything between $0< p\leq 1$

To take it one step further - remember that the key requirement for confounding adjustment is that in the pseudo-population, the probability of treatment $A$ does not depend on $L$. We can achieve this by giving everyone the same probability $p$ but we can also give different probabilities as long as the probability of treatment does not depend on $L$.

Thus, a common weight configuration is:
$SW^A=f(A)/f(A|L)$$

(For more clarity, this would mean $Pr[A=1]$ as the denominator for those who are in treatment and $Pr[A=0]$ for those who are in control.

$SW^A$ is rferred to as *stabilized weights*. The mean of the stabilized weights is expected to be 1 because the size of the pseudo-population equals that of the study population. I believe that for nonstabilized weights, the mean is 2.

In data analyses one should check that the estimated weights $SW^A$ have mean 1. Deviation indicate model misspecification or possible violations or near violoations of positivity.

Now the elephant in the room - why use stabilized weights? Because they typically result in narrower 95% CI's. However, the statistical superiority of the stabilized weights can only occur when the IP weighted model is not saturated. Although I don't think this is really an issue in most cases...

The next section describes the use of stabilized weights for a continuous treatment.


## 12.4 Marginal Structural Models

Consider the following linear model for the mean outcome under treatment level $a$
$$E[Y^a]=\beta_0+\beta_1a$$

This model differs from the previous models discussed because the outcome variable is a counterfactual, and hence generally unobserved.

Models for the marginal mean of a counterfactual outcome are referred to as *marginal structural mean models*, with the above one being a saturated one for a dichotomous treatment $A$.

The parameters for treatment in structural mean models correspond to average causal effects. Thus, $\beta_1$ above is equal to $E[Y^{a=1}]-E[Y^{a=0}]$.

A desirable property of marginal structural models is *null preservation*: When the null hypothesis of no average causal effect is true, a marginal structural model is never misspecified. For example, under this null hypothesis, the marginal structural model $E[Y^a]=\beta_0+\beta_1a+\beta_2a^2$ is correctly specified with $\beta_1=\beta_2=0$ and $\beta_0=E[Y^a]$ for any $a$. If conditional exchangeability holds, then $E[Y]=\beta_0$.

Because tretment $A$ can be polytomous or continuous, a saturated model can become impractical. This means we have to incorporate functional form assumptions. For a dichotomous treatment $A$, IP Weighting is straightforward, but for continuous treatment $A$, using IP weighting is often dangerous and the effect estimates may be exquisitely sensitive to the chioce of the model or algorithm used to estimate the conditional density $f(A|L)$. The development of methods for more stable estimation of IP weights is an active area of research.

Lastly, we can even consider a marginal structural model for a dichotomous outcome:
$$\text{logit }Pr[D^a=1]=\alpha_0+\alpha_1a$$

where $\exp(\alpha_1)$ is the causal odds ratio. The parameters are consistently estimated by fitting a logistic model to the pseudo-population created by IP weighting.

## 12.5 Effect modification and marginal structural models

MSM's do not include covariates when the target parameter is the average causal effect in the population. However, one may include covariates - which may be non-confounders - in a MSM to assess effect modification. Suppose it is hypothesized that the treatment effect varies by $V$. Then,
$$E[Y^a|V]=\beta_0+\beta_1a+\beta_2Va+\beta_3V$$

Additive effect modification is present if $\beta_2\neq 0$.

Note that the parameter $\beta_3$ generally does not have a causal interpretation, since you are assuming causal identification for treatment $A$, not $V$.

Generally, $V$ will be included in the vector of covariates $L$. Even if $V$ is not needed to ensure exchangeability, including $V$ in $L$ will generally increase the efficiency with which the parameters of the MSM are estimated.

For the IP weights,
$$SW^A(V)=\frac{f[A|V]}{f[A|L]}$$
generally result in narrower CI's around the effect estimates.

## 12.6 Censoring and Missing Data

In the case where you remove users who were censored (left before outcome), this may introduce selection bias.

Let censoring $C$ be binary flag for a user that was censored.

Because censoring due to loss to follow-up can introduce selection bias, we are generally interested in the causal effect if nbody in the study population had been censored.

This causal effect that incorporates censoring can be estimated by using IP weights:
$$W^{A,C}=W^A\times W^C$$
in which
$$W^C=1/Pr[C=0|L,A]$$ for the uncensored individuals and 0 for the censored individuals.

Then, this new weight adjusts for both confounding and selection bias under the identifiability conditions of exchangeability for the joint treatment $(A,C)$ conditional on $L$:
$$Y^{a,c=0}\perp (A,C)|L$$

There also exist stabilized versions $$SW^{A,C}=SW^A\times SW^C$$
where
$$SW^C=Pr[C=0|A]/Pr[C=0|L,A]$$

In the next chapter, we describe an alternative to IP weighting to adjust for confounding and selection bias: standardization.